## Representation learning for protein data

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyfaidx import Fasta
genes = Fasta('data/training.fasta')

In [3]:
proteins = []
for a in genes:
    proteins.append(str(a))

In [4]:
print("number of proteins",len(proteins))

number of proteins 552884


In [5]:
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y']

## Generate possible combinations of amino acids

In [6]:
from itertools import product
twograms = map(''.join, product(amino_acids, repeat=2))
threegrams = map(''.join, product(amino_acids, repeat=3))
fourgrams = map(''.join, product(amino_acids, repeat=4))
fivegrams = map(''.join, product(amino_acids, repeat=5))

In [7]:
twogramsList = {}
i = 0
for word in twograms:
    twogramsList[word] = i
    i = i + 1
    
threegramsList = {}
i = 0
for word in threegrams:
    threegramsList[word] = i
    i = i + 1

fourgramsList = {}
i = 0
for word in fourgrams:
    fourgramsList[word] = i
    i = i + 1

fivegramsList = {}
i = 0
for word in fivegrams:
    fivegramsList[word] = i
    i = i + 1

In [8]:
print(len(twogramsList))
print(len(threegramsList))
print(len(fourgramsList))
print(len(fivegramsList))

400
8000
160000
3200000


### Build training data for autoencoders

We can not build 500k X 3.2m matrix locally. So we will train small autoencoders for small batches

In [13]:
# Model paramters
n = 2
feature_batch = 400
n_record_batches = 1
gramModel = twogramsList
numOfRecords = int(100000/n_record_batches)
n_feature_batches = int(len(gramModel)/feature_batch)

# Autoencoder parameters
encoding_dim = 50
n_epochs = 10
batch_size = 256

In [14]:
from nltk import ngrams,FreqDist
twogramsDF = np.zeros([100000,feature_batch])
rowId = 1

tempgramList = {k: v for k, v in gramModel.items() if v < feature_batch}

for sentence in proteins[1:numOfRecords]:
    grams = ngrams(sentence, n)
    myFd = FreqDist(grams)
    for x in myFd:
        a = "".join(x)
        if a in tempgramList:
            twogramsDF[rowId,tempgramList[a]] = myFd[x]/ len(sentence)
    rowId = rowId + 1
    if rowId % 1000 == 0:
        print(rowId)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


## Lets train the autoencoder

In [15]:
from keras.layers import Input, Dense
from keras.models import Model

x_train = twogramsDF

print(x_train.shape)
decoding_dim = x_train.shape[1]

protein_input = Input(shape=(x_train.shape[1],))

encoded = Dense(encoding_dim, activation='relu',)(protein_input)
decoded = Dense(decoding_dim, activation='sigmoid')(encoded)

autoencoder = Model(input=protein_input, output=decoded)

autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

print()
print("\t Autoencoder training started: ")

autoencoder.fit(x_train, x_train,
                nb_epoch=n_epochs,
                batch_size=50,
                shuffle=True,validation_split=0.20)

(100000, 400)

	 Autoencoder training started: 
Train on 80000 samples, validate on 20000 samples
Epoch 1/10
80000/80000 [==============================] - 3s - loss: 0.0558 - val_loss: 8.8446e-04
Epoch 2/10
80000/80000 [==============================] - 4s - loss: 4.7119e-04 - val_loss: 2.7052e-04
Epoch 3/10
80000/80000 [==============================] - 3s - loss: 2.0007e-04 - val_loss: 1.5160e-04
Epoch 4/10
80000/80000 [==============================] - 4s - loss: 1.2478e-04 - val_loss: 1.0418e-04
Epoch 5/10
80000/80000 [==============================] - 4s - loss: 9.0487e-05 - val_loss: 7.9440e-05
Epoch 6/10
80000/80000 [==============================] - 4s - loss: 7.1273e-05 - val_loss: 6.4529e-05
Epoch 7/10
80000/80000 [==============================] - 4s - loss: 5.9161e-05 - val_loss: 5.4688e-05
Epoch 8/10
80000/80000 [==============================] - 4s - loss: 5.0915e-05 - val_loss: 4.7774e-05
Epoch 9/10
80000/80000 [==============================] - 4s - loss: 4.4988e-05 - 

I think mean squared error might be misleading as it is too low. It seems like we are not learning anything. Lets try R2 square error to see if we are actually alearning anything

## Autoencoder with R2 squared error

In [16]:
from keras import backend as K
def r2_score_(y_true, y_pred):
    #print(y_pred)
    num = K.mean(K.square(y_pred - y_true), axis=-1) 
    denom = K.mean(K.square(y_true - K.mean(y_true)), axis=-1) 
    return (num/denom)

In [18]:
from keras.layers import Input, Dense
from keras.models import Model

x_train = twogramsDF

print(x_train.shape)
decoding_dim = x_train.shape[1]

protein_input = Input(shape=(x_train.shape[1],))

encoded = Dense(encoding_dim, activation='relu',)(protein_input)
decoded = Dense(decoding_dim, activation='sigmoid')(encoded)

autoencoder = Model(input=protein_input, output=decoded)

autoencoder.compile(optimizer='adadelta', loss=r2_score_,metrics=['mean_squared_error'])

print()
print("\t Autoencoder training started: ")

autoencoder.fit(x_train, x_train,
                nb_epoch=n_epochs,
                batch_size=50,
                shuffle=True,validation_split=0.20)

(100000, 400)

	 Autoencoder training started: 
Train on 80000 samples, validate on 20000 samples
Epoch 1/10
80000/80000 [==============================] - 4s - loss: 345.8333 - mean_squared_error: 0.0042 - val_loss: 0.7893 - val_mean_squared_error: 1.5581e-05
Epoch 2/10
80000/80000 [==============================] - 4s - loss: 0.7769 - mean_squared_error: 1.5289e-05 - val_loss: 0.7866 - val_mean_squared_error: 1.5514e-05
Epoch 3/10
80000/80000 [==============================] - 4s - loss: 0.7739 - mean_squared_error: 1.5230e-05 - val_loss: 0.7841 - val_mean_squared_error: 1.5459e-05
Epoch 4/10
80000/80000 [==============================] - 4s - loss: 0.7702 - mean_squared_error: 1.5158e-05 - val_loss: 0.7811 - val_mean_squared_error: 1.5396e-05
Epoch 5/10
80000/80000 [==============================] - 4s - loss: 0.7637 - mean_squared_error: 1.5028e-05 - val_loss: 0.7738 - val_mean_squared_error: 1.5260e-05
Epoch 6/10
80000/80000 [==============================] - 4s - loss: 0.7527 - m

We can clearly see that we are able to reduce r2 error on both training and validation data. Let's increase epochs to see how far we can see the improvements in r2 error

### Epoch set to 50

In [19]:
# Overridden
n_epochs = 50

In [20]:
from keras.layers import Input, Dense
from keras.models import Model

x_train = twogramsDF

print(x_train.shape)
decoding_dim = x_train.shape[1]

protein_input = Input(shape=(x_train.shape[1],))

encoded = Dense(encoding_dim, activation='relu',)(protein_input)
decoded = Dense(decoding_dim, activation='sigmoid')(encoded)

autoencoder = Model(input=protein_input, output=decoded)

autoencoder.compile(optimizer='adadelta', loss=r2_score_,metrics=['mean_squared_error'])

print()
print("\t Autoencoder training started: ")

autoencoder.fit(x_train, x_train,
                nb_epoch=n_epochs,
                batch_size=50,
                shuffle=True,validation_split=0.20)

(100000, 400)

	 Autoencoder training started: 
Train on 80000 samples, validate on 20000 samples
Epoch 1/50
80000/80000 [==============================] - 4s - loss: 354.6874 - mean_squared_error: 0.0043 - val_loss: 0.7892 - val_mean_squared_error: 1.5582e-05
Epoch 2/50
80000/80000 [==============================] - 4s - loss: 0.7766 - mean_squared_error: 1.5285e-05 - val_loss: 0.7862 - val_mean_squared_error: 1.5506e-05
Epoch 3/50
80000/80000 [==============================] - 4s - loss: 0.7732 - mean_squared_error: 1.5218e-05 - val_loss: 0.7837 - val_mean_squared_error: 1.5452e-05
Epoch 4/50
80000/80000 [==============================] - 4s - loss: 0.7688 - mean_squared_error: 1.5131e-05 - val_loss: 0.7788 - val_mean_squared_error: 1.5351e-05
Epoch 5/50
80000/80000 [==============================] - 4s - loss: 0.7607 - mean_squared_error: 1.4966e-05 - val_loss: 0.7700 - val_mean_squared_error: 1.5179e-05
Epoch 6/50
80000/80000 [==============================] - 4s - loss: 0.7489 - m

We are able to reduce R2 error to almost half. Similary I have tested autoencoders on 3,4,5 gram models. On higher gram models its performance is not good. I will try to use deep auto encoders to see if performance improves.